In [1]:
%%capture

pip install solana

In [2]:
import os
import json
import time
from solana.keypair import Keypair
from solana.publickey import PublicKey
from solana.rpc.api import Client
from solana.transaction import Transaction
from solana.system_program import TransferParams, transfer

In [3]:
solana_client = Client("https://api.devnet.solana.com")

In [4]:
def pp(x):
    if type(x) == dict:
        print(json.dumps(x, indent=2))
    else:
        print(json.dumps(json.loads(x), indent=2))

In [5]:
def create_account(sender_username):
    try:
        kp = Keypair.generate()
        public_key = str(kp.public_key)
        secret_key = kp.secret_key

        data = {
            'public_key': public_key,
            'secret_key': secret_key.decode("latin-1"),
        }
        if "solana" not in os.listdir("./"):
            os.mkdir("./solana/")
        file_name = './solana/{}.txt'.format(sender_username)
        with open(file_name, 'w') as outfile:
            json.dump(data, outfile)

        return public_key
    except Exception as e:
        print('error:', e)
        return None

In [6]:
uname = 'testuser'

In [7]:
create_account(uname)

'CAPX9Yz7qiKc6Mka92fT3tV3wF5tNKHCeThGCYJ8FHTd'

In [8]:
def load_wallet(sender_username):
    try:
        file_name = './solana/{}.txt'.format(sender_username)
        with open(file_name) as json_file:
            account = json.load(json_file)
            account['secret_key'] = account['secret_key'].encode("latin-1")
            return account

    except Exception as e:
        print(e)
        return None  

In [9]:
load_wallet(uname)

{'public_key': 'CAPX9Yz7qiKc6Mka92fT3tV3wF5tNKHCeThGCYJ8FHTd',
 'secret_key': b'l\xafaD\xeb\xb9\xab%p\xd1\xeaOc\xaeo\x08\x8f\xa6Q\x88$^\xe6\x90\x00A\xfb\xe8u\x85\xe1\xde\xa5\xd8?%\xe2\xfd\xe7\xa3\xb7\xe8\xf4\xf6\xc1\xa8\xb9\xd8\x0e\x0f{\xb76\xb6q\xcb\xf3\x07;\xb3\xf5Me\xc8'}

In [10]:
def fund_account(sender_username: str, amount: int):
    try:
        amount = int(1000000000 * amount)
        account = load_wallet(sender_username)
        
        resp = solana_client.request_airdrop(
            account['public_key'], amount
        )
        print(resp)    

        transaction_id = resp['result']
        if transaction_id != None:
            return transaction_id
        else:
            return None

    except Exception as e:
        print('error:', e)
        return None


In [11]:
for i in range(5):
    fund_account(uname, 1)
    time.sleep(10) # Need to wait otherwise it'll error out

{'jsonrpc': '2.0', 'result': '4eAfrxoEhfGJkgJHV5WviE6gV8TensJHnHJNZUYRoK71qGfjwFT3d6aCi55o3VFizuTnDh8fPKB5rzCWiN761fJC', 'id': 1}
{'jsonrpc': '2.0', 'result': '4D2wPm7goDPQo9mZD2hU5FLvkjcW1GPcUJmrVafDVFs2ixpFTofBXFu9thsENHuF5XXurccj35TgvS6iWcWqvTtf', 'id': 2}
{'jsonrpc': '2.0', 'result': '2zjc8XAXpH7v6H6Lunti2mrrs3h4xuSLPA5WuZij5NiRhhmYUQKTEmShKrGtojUkZu5JYwpPNVWRFBuNc3ti4JHc', 'id': 3}
{'jsonrpc': '2.0', 'result': '5nr9duUyyuSaiuCojzGH8D8Gup9PE4xCraqSNX7JdkKmodKqRjvpHfcjhxoHguXiT8W5DJppik8PEQxr4VjgjUoE', 'id': 4}
{'jsonrpc': '2.0', 'result': '5yaCdNfiWoCe9SBAbFV1SeNjKns5AWbR4L3c1fmar7KPGT6PMjdAphaYkXhN4z4BC56BA6vbs1r1f12SSWdaxDBo', 'id': 5}


In [12]:
def get_balance(sender_username: str):
    try:
        account = load_wallet(sender_username)
        resp = solana_client.get_balance(account['public_key'])
        pp(resp)
        balance = resp['result']['value'] / 1000000000
        data = {
            "publicKey": account['public_key'],
            "balance": str(balance),
        }
        return data
    except Exception as e:
        print('error:', e)
        return None

In [13]:
r = get_balance(uname)

{
  "jsonrpc": "2.0",
  "result": {
    "context": {
      "slot": 123651066
    },
    "value": 4000000000
  },
  "id": 6
}


In [14]:
pp(r)

{
  "publicKey": "CAPX9Yz7qiKc6Mka92fT3tV3wF5tNKHCeThGCYJ8FHTd",
  "balance": "4.0"
}


In [15]:
def send_sol(sender_username, amount, receiver):
    try:
        account = load_wallet(sender_username)
        sender = Keypair.from_secret_key(account['secret_key'])
        amount = int(1000000000 * amount)

        txn = Transaction().add(
            transfer(
                TransferParams(
                    from_pubkey=sender.public_key, 
                    to_pubkey=PublicKey(receiver), 
                    lamports=amount
                )
            )
        )
        resp = solana_client.send_transaction(txn, sender)
        pp(resp)

        transaction_id = resp['result']
        if transaction_id != None:
            return transaction_id
        else:
            return None

    except Exception as e:
        print('error:', e)
        return None

In [16]:
uname2 = 'testuser2'

In [17]:
create_account(uname2)

'BiTanY66aBrFQSG9dNVZtJj8ssmacNRVZuRLyTyZyWam'

In [18]:
account2 = load_wallet(uname2)

In [19]:
send_sol(uname, 1, account2['public_key'])

{
  "jsonrpc": "2.0",
  "result": "5DuiocHAQFnA3NbmEh6EiBtHiwakB4vkrPp33N2rWpBcCz5LkmCHSZbv133rjC6C1jHr8rhi1g3eiydNRR9JP27D",
  "id": 8
}


'5DuiocHAQFnA3NbmEh6EiBtHiwakB4vkrPp33N2rWpBcCz5LkmCHSZbv133rjC6C1jHr8rhi1g3eiydNRR9JP27D'

In [21]:
txsig = solana_client.get_transaction(tx_sig='5DuiocHAQFnA3NbmEh6EiBtHiwakB4vkrPp33N2rWpBcCz5LkmCHSZbv133rjC6C1jHr8rhi1g3eiydNRR9JP27D')

In [22]:
pp(txsig)

{
  "jsonrpc": "2.0",
  "result": {
    "blockTime": 1648293211,
    "meta": {
      "err": null,
      "fee": 5000,
      "innerInstructions": [],
      "logMessages": [
        "Program 11111111111111111111111111111111 invoke [1]",
        "Program 11111111111111111111111111111111 success"
      ],
      "postBalances": [
        3999995000,
        1000000000,
        1
      ],
      "postTokenBalances": [],
      "preBalances": [
        5000000000,
        0,
        1
      ],
      "preTokenBalances": [],
      "rewards": [],
      "status": {
        "Ok": null
      }
    },
    "slot": 123651130,
    "transaction": {
      "message": {
        "accountKeys": [
          "CAPX9Yz7qiKc6Mka92fT3tV3wF5tNKHCeThGCYJ8FHTd",
          "BiTanY66aBrFQSG9dNVZtJj8ssmacNRVZuRLyTyZyWam",
          "11111111111111111111111111111111"
        ],
        "header": {
          "numReadonlySignedAccounts": 0,
          "numReadonlyUnsignedAccounts": 1,
          "numRequiredSignatures": 1
     

Easy to view this transaction on Solana at this URL:

https://explorer.solana.com/tx/5DuiocHAQFnA3NbmEh6EiBtHiwakB4vkrPp33N2rWpBcCz5LkmCHSZbv133rjC6C1jHr8rhi1g3eiydNRR9JP27D?cluster=devnet

# End